# TensorFlow 2 quickstart

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

## Set up TensorFlow

Import TensorFlow into your program:

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 0s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model:

In [6]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

The model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [7]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.33801585, -0.575005  , -0.00572306, -0.4033052 , -0.26488456,
         0.5333041 , -0.26540297,  0.36734414,  0.7524033 , -0.37305576]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [8]:
tf.nn.softmax(predictions).numpy()

array([[0.06836733, 0.05394182, 0.09531506, 0.06404626, 0.07355449,
        0.16340226, 0.07351635, 0.13841482, 0.20342843, 0.06601323]],
      dtype=float32)

It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [14]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [15]:
loss_fn(y_train[:1], predictions).numpy()

1.8115404

Before training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [16]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate the model

Use the `Model.fit` method to adjust the model parameters and minimize the loss:

In [17]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.2982 - accuracy: 0.9133
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.1450 - accuracy: 0.9571
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1082 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0861 - accuracy: 0.9736
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0738 - accuracy: 0.9769


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [18]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0764 - accuracy: 0.9757 - 658ms/epoch - 2ms/step


[0.07642917335033417, 0.9757000207901001]

The image classifier is now trained to ~98% accuracy on this dataset.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [19]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [20]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.1558096e-08, 4.9750526e-10, 1.7991318e-06, 2.7637698e-05,
        4.6512703e-11, 2.7806379e-08, 4.5994575e-15, 9.9996412e-01,
        4.2299362e-07, 5.9783370e-06],
       [7.9960142e-07, 4.1699648e-04, 9.9957067e-01, 5.9193485e-06,
        8.4378680e-16, 5.3352669e-06, 2.4255164e-07, 8.9248045e-12,
        6.9482566e-09, 4.2386109e-13],
       [6.7253563e-06, 9.9738497e-01, 3.1294042e-04, 1.3642311e-04,
        2.7612090e-05, 2.0062091e-05, 3.7052341e-05, 1.7906828e-03,
        2.7443509e-04, 9.0986250e-06],
       [9.9999571e-01, 1.7790008e-10, 5.2520761e-07, 2.9077114e-09,
        5.6853156e-10, 2.6677753e-07, 1.9748243e-06, 1.3105465e-06,
        1.8981973e-08, 2.1564202e-07],
       [1.9868644e-06, 1.3744500e-09, 4.7074741e-06, 1.4545668e-08,
        9.9161083e-01, 6.3073480e-07, 1.1812903e-07, 9.2271739e-06,
        1.2937037e-06, 8.3711576e-03]], dtype=float32)>